# Imports

In [1]:
from utils import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import numpy as np
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

C:\Users\Joely\anaconda3\envs\comp_genomics\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Joely\anaconda3\envs\comp_genomics\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


# Data Loading

In [2]:
seqs, labels = load_data_train()

# Model

### Experimentation Setup

In [3]:
curr_cutoff = None

In [4]:
# experiments
# NOTE THAT in_channels_conv1 is always the seq_cutoff length
hyperparams = [
    # EXPERIMENT 0
    {'in_channels_conv1': 1000,     'out_channels_conv1': 300,
     'in_channels_conv2': 150,     'out_channels_conv2': 300,
     'in_channels_conv3': 150,     'out_channels_conv3': 300,
     'in_channels_fc1': 150,     'in_channels_fc2': 300, 
     'kernel_size': 20,     'stride': 20,
     'decay': 0.01,
     'epochs': 10,
     'train_acc': 0.33,     'test_acc': 0.33,     'time': None
    },
    
    # EXPERIMENT 1
    {'in_channels_conv1': 2000,     'out_channels_conv1': 600,
     'in_channels_conv2': 300,     'out_channels_conv2': 600,
     'in_channels_conv3': 300,     'out_channels_conv3': 600,
     'in_channels_fc1': 300,     'in_channels_fc2': 600, 
     'kernel_size': 20,     'stride': 20,
     'decay': 0.01,
     'epochs': 10,
     'train_acc': 0.36,     'test_acc': 0.34,     'time': "5hr"
    },  
    
    # EXPERIMENT 2
    {'in_channels_conv1': 1000,     'out_channels_conv1': 300,
     'in_channels_conv2': 150,     'out_channels_conv2': 75,
     'in_channels_conv3': 75//2,     'out_channels_conv3': 75//2,
     'in_channels_fc1': 38,     'in_channels_fc2': 300, 
     'kernel_size': 20,     'stride': 20,
     'decay': 0.01,
     'epochs': 10,
     'train_acc': 0.27,     'test_acc': 0.27,     'time': "1h 1m 40s"
    },
]

i = 2
hp = hyperparams[i]

In [5]:
if curr_cutoff is None or curr_cutoff != hp['in_channels_conv1']:
        train_dataloader, test_dataloader, new_seqs, voc2ind, tokenized_seqs, labels = prep_data(seqs, labels, cutoff_len = hp['in_channels_conv1'])
        FEATURE_SIZE = tokenized_seqs.shape[1]
else:
    print('already prepared for this in_channels_conv1')
    
curr_cutoff = hp['in_channels_conv1']

data prepared
Finding Max Sequence Length
Max sequence length found: 1000
Min sequence length found: 20
Begin Padding
Change to a numpy array
tokenized sequenced
Begin prepare labels
tokenized labels


 [[1. 2. 3. ... 2. 2. 4.]
 [4. 1. 3. ... 0. 0. 0.]
 [5. 5. 1. ... 4. 4. 4.]
 ...
 [1. 1. 4. ... 1. 4. 1.]
 [1. 3. 2. ... 4. 2. 1.]
 [3. 4. 4. ... 1. 2. 1.]] 

 {'<pad>': 0, 'C': 1, 'A': 2, 'T': 3, 'G': 4, 'N': 5} 

 {'RYUA3GVO': 51, 'UK4B4I7A': 23, 'YMHGXK99': 422, '9HRDSOST': 15, 'L905DK46': 5, 'YKXRSB4N': 100, 'I7FXTVDP': 8286, 'FRX9XJYW': 261, 'JO1WTZOB': 28, 'THW6JGC7': 133, 'B2BULVFH': 23, '3N169DM2': 6, '3EARN0Z7': 5, 'Q1D88JO2': 6, 'HVXSID0M': 31, 'W1STLS0T': 456, '5PC2F8NE': 34, '97PR85CP': 23, 'MQQTIYIC': 10, 'ZT1IP3T6': 26, '4PKCMX7O': 25, '3TUFYWQN': 9, '9U0DELRD': 61, 'ZB862XHR': 28, 'H9RBDN30': 20, 'NMQKJMH3': 16, 'T8R673OI': 8, 'EYOJGC9T': 98, '1B9BJ2IP': 7, 'Y5YH740Y': 7, 'REKW7MRF': 13, 'F3D2JAYU': 43, '7QF2VB5B': 166, 'QYBCIW4J': 5, '3MDRJUI2': 12, '459BZKP3': 91, 'IZD0O

In [ ]:
class DSC_by_CNN(nn.Module):
    def __init__(self, vocab_size, feature_size, num_classes):
        


        super(DSC_by_CNN, self).__init__()
        # Number of vocab words
        self.vocab_size = vocab_size
        # embeded_dim: Dimension of word vectors.
        self.feature_size = feature_size

        self.encoder = nn.Embedding(self.vocab_size, self.feature_size)
        

        self.conv1 = nn.Conv1d(in_channels=hp['in_channels_conv1'],
                               out_channels=hp['out_channels_conv1'], 
                               kernel_size=hp['kernel_size'],
                               stride = hp['stride'],
                               padding = 1)
        
        self.conv2 = nn.Conv1d(in_channels=hp['in_channels_conv2'],
                               out_channels=hp['out_channels_conv2'], 
                               kernel_size=hp['kernel_size'],
                               stride = hp['stride'],
                               padding = 1)
        
        self.conv3 = nn.Conv1d(in_channels=hp['in_channels_conv3'],
                               out_channels=hp['out_channels_conv3'], 
                               kernel_size=hp['kernel_size'],
                               stride = hp['stride'],
                               padding = 1)

        # Activation
        self.act = nn.ReLU()

        # Pooling
        self.pool = nn.MaxPool2d(3, 2, 1)

        # fully connected layer
        self.fc1 = nn.Linear(hp['in_channels_fc1'], hp['in_channels_fc2'])
        self.fc2 = nn.Linear(hp['in_channels_fc2'], num_classes)

        # dropout
        self.drop = nn.Dropout(p=0.5)


    # seen in the network pytorch tutorial
    # I've been using this function for years, no idea what it does.
    # Would it kill the pytorch tutorial people to document this guy?
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features        


    def forward(self, x):
        # print("forward")
        # The following line is required to send it through the encoder. 
        # because indexes are neccesary for use in the encoder
        # x = torch.tensor(x).to(torch.int64)
        x = x.clone().detach().requires_grad_(True).to(torch.int64)
        
    
        # Encode. 
        # Output shape: (b, max_len, embedded_dim)
        x = self.encoder(x)
  
        # Permute shuffles dimensions. Do this to satisfy requirments of nn.Conv1d
        # Output shape: (b, embedded_dim, max_len)
        x = x.permute(0, 2, 1)

        # convolutional layer 1
        x = self.conv1(x)
        x = self.act(x)                         
        x = self.pool(x)

        # convolutional layer 2
        x = self.conv2(x)
        x = self.act(x)
        x = self.pool(x)
        
        # convolutional layer 3

        # flattening
        x = x.view(-1, self.num_flat_features(x))

        # Tring to figure out the shape
        #print(x.shape)

        # fully linear layer + dropout
        x = self.drop(x)
        # x = self.softm(x)
        x = self.fc1(x)
        
        # fully linear layer
        x = self.fc2(x)

        return x
    
start_time = time.time()
# Creating net and training
net = DSC_by_CNN(len(voc2ind), FEATURE_SIZE, len(set(labels)))
losses = train(net, train_dataloader, device, epochs=hp['epochs'], lr=0.01, decay = hp['decay'])


# Evaluation
plot_losses(losses, title = "Splice Dataset: Convolutional Network Loss")
print_eval(net, train_dataloader, test_dataloader)
print()

# time to complete
end_time = time.time()
print('Time to complete', time_str(end_time - start_time))